In [1]:
import sys
sys.path.append(f"tformer") # add tformer folder

from train import train_model, test_model
import warnings

warnings.filterwarnings("ignore")

In [5]:
def get_config():
    return {
        "batch_size": 1,   # 1: ~6G / For A6000(48G)
        "num_epochs": 30,
        "lr": 10**-4,
        "seq": 1300,
        "d_model": 512,
        "datasource": 'lemon-mint/korean_english_parallel_wiki_augmented_v1',
        "lang_src": "en",
        "lang_src_fieldname": "english",
        "lang_tgt": "kr",
        "lang_tgt_fieldname": "korean",
        "model_folder": "lemon-mint/weights",
        "model_basename": "tmodel_",
        "preload": "latest",
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

config = get_config()

### Training

In [6]:
train_model(config)

Using device: cuda
Device name: NVIDIA RTX A6000
Device memory: 47.988 GB


Processing Max Length(encode): 100%|██████████| 50324/50324 [00:17<00:00, 2796.13it/s]


Max length of source sentence: 1286
Max length of target sentence: 937
Preloading model lemon-mint\weights\tmodel_29.pt


### Evaluating

In [5]:
import torch
import gc

def clear_all_cuda_memory(verbose=True):
    """
    현재 스코프 내의 모든 GPU 할당 변수를 삭제하고,
    CUDA 캐시 및 Python 가비지 컬렉션을 실행하여 GPU 메모리를 최대한 정리합니다.
    """
    if not torch.cuda.is_available():
        if verbose:
            print("CUDA를 사용할 수 없습니다.")
        return

    if verbose:
        print(f"클리어 전 할당된 GPU 메모리: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
        print(f"클리어 전 캐시된 GPU 메모리: {torch.cuda.memory_cached() / (1024**2):.2f} MB")

    # 1. 현재 스코프 내의 모든 GPU 텐서 변수를 찾아 삭제
    # 이 부분은 현재 실행 환경(예: Jupyter Notebook 셀)에 따라 동작 방식이 다를 수 있습니다.
    # 일반적으로는 명시적으로 변수를 None으로 설정하거나 del 하는 것이 가장 확실합니다.
    # 아래 코드는 일반적인 상황에서 참조를 끊는 데 도움이 됩니다.
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                if obj.is_cuda:
                    del obj
        except:
            pass # 예외 처리 (e.g., 객체가 이미 삭제된 경우)

    # 2. PyTorch CUDA 캐시 비우기
    torch.cuda.empty_cache()

    # 3. Python 가비지 컬렉션 강제 실행
    gc.collect()

    if verbose:
        print(f"클리어 후 할당된 GPU 메모리: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
        print(f"클리어 후 캐시된 GPU 메모리: {torch.cuda.memory_cached() / (1024**2):.2f} MB")
        print("CUDA 메모리가 완전히 정리되었습니다.")

clear_all_cuda_memory(verbose=True)

클리어 전 할당된 GPU 메모리: 1074.15 MB
클리어 전 캐시된 GPU 메모리: 1092.00 MB
클리어 후 할당된 GPU 메모리: 0.00 MB
클리어 후 캐시된 GPU 메모리: 1092.00 MB
CUDA 메모리가 완전히 정리되었습니다.


In [17]:
test_model(config)

Using device: cuda
Device name: NVIDIA RTX A6000
Device memory: 47.988 GB


Processing Max Length(encode): 100%|██████████| 50324/50324 [00:16<00:00, 3063.96it/s]


Max length of source sentence: 1286
Max length of target sentence: 937
Preloading model lemon-mint\weights\tmodel_00.pt
--------------------------------------------------------------------------------
    SOURCE: The United States Unified Medical Command was a proposed Unified Combatant Command within the United States Department of Defense. The plan was rejected in December 2006 due to heavy resistance from Air Force Surgeon General Lieutenant General James G. Roudebush, who cited mission and cultural differences among the services as justification for keeping medical assets separate.

The unified command would have merged the medical assets of the United States Navy, Air Force, and Army and reduced the duplication of effort that currently exists.
    TARGET: 미국 통합 의료 사령부는 미국 국방부 내에 설립될 예정이었던 통합 전투 사령부였습니다. 하지만 2006년 12월 공군 군의관 총장 제임스 G. 루데부시 중장의 강력한 반대로 계획이 폐기되었습니다. 루데부시 중장은 임무와 문화적 차이를 이유로 의료 자산을 분리해야 한다고 주장했습니다.

통합 사령부는 미국 해군, 공군, 육군의 의료 자산을 통합하여 현재 존재하는 중복 노력을 줄이려는 목표를 가지고 있었습니다.